## GPT函数调用

In [1]:
def get_current_weather(location="北京市"):
    return "多云，温度26摄氏度"


In [2]:
from openai import OpenAI
client = OpenAI()

In [9]:
def chat_completion_request(messages, tools=None, tool_choice=None):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo-0613",
            messages=messages,
            tools=tools,
            tool_choice=tool_choice
        )
        return response
    except Exception as e:
        print(f"exception: {e}")
        return e

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "获取当前的天气",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市名称，比如北京",
                    },
                },
                "required": ["location"],
            },
        }
    },
]

In [19]:
messages = []
messages.append({"role": "system", "content": "回答用户的问题，并给出相关出行建议"})
messages.append({"role": "user", "content": "今天深圳的天气怎么样"})
chat_response = chat_completion_request(
    messages, tools=tools
)

assistant_message = chat_response.choices[0].message
# print(assistant_message)

In [12]:
assistant_message.model_dump()

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'call_xOZKKfJ9ht5zDs2XJeAXG7w0',
   'function': {'arguments': '{\n  "location": "深圳"\n}',
    'name': 'get_current_weather'},
   'type': 'function'}]}

In [13]:
def execute_func_call(messages):
    if messages.tool_calls[0].function.name == "get_current_weather":
        argument = messages.tool_calls[0].function.arguments
        results = get_current_weather(argument)
        return results
    else:
        return f"function does not exist"

In [20]:
if assistant_message.tool_calls:
    results = execute_func_call(assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, 
                     "name": assistant_message.tool_calls[0].function.name, 
                     "content": results})

assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})

print(messages)

[{'role': 'system', 'content': '回答用户的问题，并给出相关出行建议'}, {'role': 'user', 'content': '今天深圳的天气怎么样'}, {'role': 'function', 'tool_call_id': 'call_O66csPz01nfAkAi87z4618TB', 'name': 'get_current_weather', 'content': '多云，温度26摄氏度'}, {'role': 'assistant', 'content': 'Function(arguments=\'{\\n  "location": "深圳"\\n}\', name=\'get_current_weather\')'}]


In [21]:
chat_response = chat_completion_request(
    messages, tools=None, tool_choice=None
)
print(chat_response.choices[0].message.content)

根据最新的天气信息，深圳今天多云，气温约为26摄氏度。如果您计划外出，建议您带上一把雨伞，以防突然下雨。另外，注意防晒并保持水分补充，确保身体健康。祝您有个愉快的一天！
